## Evolutionary algorithm (USPEX) for target XRD csv file (with DFT (ex- VASP code))

### Setting for USPEX in local PC <->VASP in supercomputer connection without typing password

This procedure is necessary for using USPEX with DFT structural optimization commonly performed in another cluster machine.

Make key of computer with USPEX
> ssh-keygen -t rsa -q -f ~/.ssh/id_rsa -N ""

Copy the generated key to target supercomputer with VASP code (in case the generated key was not registered in ~/.ssh/authorized_keys in host supercomputer)
> ssh-copy-id -i ~/.ssh/id_rsa.pub [id@supercomputer IP or address]

Then, type [yes], and insert password for supercomputer.<br>
After doing this, typing password is skipped to login to supercomputer.<br>
This is essential for sending and bringing VASP results automatically.<br>


### Setting for USPEX in local PC <->VASP in supercomputer connection: modifying files in uspex/src/Submission/ folder

Three files, checkStatus_remote.m, submitJob_remote.m, checkStatusC.m in Submission folder should be modified.<br>
<br>
They must be adapted for user environement. <br>
<br>
USPEX makes each generation (structure) ID and send it to supercomputer.<br>
Then, supercomputer queue system makes job ID. <br>
USPEX generation ID - supercomputer queue system job ID should be matched well. <br>
USPEX frequnetly checks that job ID is displayed as pending/running in supercomputer queue system.<br>
Then, when the job ID disappears on the queue system, USPEX judges whether the VASP calculation was finished and copy back it to USPEX.<br>
Therefore, the format of display of pending/running on queue system should be carefully checked and reflected to checkStatus_remote.m & submitJob_remote.m. <br>
Unfortunately, supercomputer queue environement are different (for example, pbs, slurm, ...), so it is needed to set individually.<br>
The followings are example:

- checkStatus_remote.m <br>
  At 10th row, Home : Insert home directory of supercomputer <br>
  At 11th row, Address : Insert [ID@supercomputer IP or address] <br>
  At 15th row : need to check qstat (display queue status) is available  (slurm case : squeue) <br>
  At 18th & 21th row : need to check that running/pending status on queue system is well captured. <br>
      It depends on type of queue system. <br>
    
    
- submitJob_remote.m
  Sun grid queue system information : https://bioinformatics.mdc-berlin.de/intro2UnixandSGE/sun_grid_engine_for_beginners/how_to_submit_a_job_using_qsub.html <br>
  Detailed modifications might be necessary. <br>
  At 26-36th row : setting for shell-script for VASP <br>
    32nd row : Specify maximum run time (short time such as 10 minutes could be better for ignoring nonsense unstable structures) <br>
    34th row : Need to check VASP is well executed. <br>
  At 51st row, Home : Insert home directory of supercomputer <br>
  At 52nd row, Address : Insert [ID@supercomputer IP or address] <br>
  At 89th row, need to check qsub is well set up.  (slurm case : sbatch) <br>
  At 100th row, need to check job ID is well captured. <br>
    It depends on type of queue system. <br>

When USPEX is running, /Submission folder is automatically copied to the USPEX-running folder. <br>
The Submission folder in USPEX-running folder is read in prior to USPEX-source folder, ~/uspex/src/Submission/. <br>



### Addiotional procedure for with VASP : check INPUT.txt
 
INPUT.txt includes various input parameters for USPEX. <br>
: same as without DFT case. Just need more VASP-related input in INPUT.txt as following:<br>
<br>

- **abinitioCode** : set 1 <br>
  0 : no DFT, 1 : VASP
  
- **KresolStart** : k-point density

- **whichCluster** : set 2 <br>
  0 : local PC, 2 : supercomputer (remote)

- **numParallelCalcs** : # of parallel calculations for supercomputer

- **remoteFolder** : remote folder at supercomputer


### Check Specific folder (additional information for VASP)

- In Specific folder, xrd target csv file is located. (xrd_target.csv) <br>
<br>
- Prepare INCAR as INCAR_1 in ./Specific folder. <br> 
If multiple VASP calculations are needed, prepare INCAR_1, INCAR_2, ..., INCAR_N, and set KresolStart in INPUT.txt with multiple numbers.<br>
 <br>
- In addition, prepare POTCARs with each element with name in ./Specific folder.  (e.g.) POTCAR_Mg, POTCAR_O   <br>
<br>


### RUN VASP : same as without DFT case


### When USPEX with VASP does not work well...

There are many patterns that USPEX does no work well.

- bad connection with supercomputer

In my experience, a bad connection with supercomputer for VASP made many problems.

In the errored case, check files in Submission folder, and remove the failed files and run again.

In addition, sometimes bad initial structure for VASP generates CONTCAR including strange lattice parameter with too large digit.

- strange CONTCAR because of structural optimization from strange initial structure

CONTCAR file have strange latice parameter matrix such as too large number without space

In that case, copy POSCAR to CONTCAR, and continue running USPEX. 

USPEX will load unoptimized CONTCAR file and go to investigate the next structure file.